In [ ]:
# --- Cell 0: Load Source Data & Auto-Detect Parameters ---
%matplotlib widget

import importlib
import sys
import traceback
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipyfilechooser import FileChooser
from IPython.display import clear_output, display

import opym
import opym.metadata  # Ensure metadata module is available

# Reload modules
try:
    importlib.reload(opym.dataloader)
    importlib.reload(opym.petakit)
    importlib.reload(opym.utils)
    importlib.reload(opym.viewer)
    importlib.reload(opym.metadata)  # Added metadata reload
    importlib.reload(opym)
    print("Reloaded opym library.")
except Exception as e:
    print(f"Note: Could not reload all modules: {e}")


# --- Create Interactive Widgets ---
print("\n1. Select your source data directory (OPM or LLSM).")
print("2. Choose the channel to use for the preview MIP.")
print("3. Click 'Load Data'.")

dir_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    title="<b>Select Source TIFF Directory (OPM or LLSM):</b>",
    show_only_dirs=True,
)

preview_channel_widget = widgets.IntText(
    value=2,
    description="Preview Channel:",
    style={"description_width": "initial"},
)

load_button = widgets.Button(description="Load Data", button_style="primary")
load_output = widgets.Output()

# --- Global variables to hold data and parameters ---
source_processed_dir: Path | None = None
mip_original: np.ndarray | None = None
stack_original: np.ndarray | None = None
vmin_original: float = 0
vmax_original: float = 1
data_type_source: opym.MicroscopyDataType = "UNKNOWN"

# --- Globals for OPM data (used in Cell 1) ---
get_stack_source = None
t_min_source = 0
t_max_source = 0
c_min_source = 0
c_max_source = 0
base_name_source = ""

# --- Default physical params (Will be updated by auto-detection) ---
default_xy_size = 0.136
default_z_step = 1.0
default_angle = 31.8


def on_load_button_clicked(b):
    global source_processed_dir, mip_original, vmin_original, vmax_original
    global get_stack_source, t_min_source, t_max_source, c_min_source
    global c_max_source, base_name_source, data_type_source
    global stack_original
    # Allow updating defaults for Cell 1
    global default_z_step, default_angle, default_xy_size

    with load_output:
        clear_output(wait=True)
        if not dir_chooser.value:
            print("❌ ERROR: No directory selected.")
            return

        source_processed_dir = Path(dir_chooser.value)
        preview_channel = preview_channel_widget.value

        try:
            # 1. Detect Data Type
            data_type_source = opym.detect_microscopy_data_type(source_processed_dir)

            if data_type_source == "OPM":
                print(f"--- Loading OPM data from: {source_processed_dir.name} ---")
                (
                    get_stack,
                    t_min,
                    t_max,
                    c_min,
                    c_max,
                    z_max,
                    y,
                    x,
                    base_name,
                ) = opym.load_tiff_series(source_processed_dir)

                # Store globals
                get_stack_source = get_stack
                t_min_source = t_min
                t_max_source = t_max
                c_min_source = c_min
                c_max_source = c_max
                base_name_source = base_name

                # Validate preview channel
                if not c_min <= preview_channel <= c_max:
                    print(
                        f"⚠️ Warning: Channel {preview_channel} not in range "
                        "({c_min}-{c_max}). Defaulting to C={c_min}."
                    )
                    preview_channel = c_min

                print(f"  Generating MIP from T={t_min}, C={preview_channel}...")
                stack_original = get_stack(t_min, preview_channel)
                mip_original = np.max(stack_original, axis=0)

            elif data_type_source == "LLSM":
                print(f"--- Loading LLSM preview from: {source_processed_dir.name} ---")
                (
                    get_stack,
                    t_min,
                    t_max,
                    c_min,
                    c_max,
                    z_max,
                    y,
                    x,
                    base_name,
                ) = opym.load_llsm_tiff_series(source_processed_dir)

                if not c_min <= preview_channel <= c_max:
                    print(
                        f"⚠️ Warning: Channel {preview_channel} not in range "
                        "({c_min}-{c_max}). Defaulting to C={c_min}."
                    )
                    preview_channel = c_min

                print(f"  Generating MIP from T={t_min}, C={preview_channel}...")
                stack_original = get_stack(t_min, preview_channel)
                mip_original = np.max(stack_original, axis=0)

                # Store globals
                get_stack_source = None
                base_name_source = base_name

            else:
                print("❌ ERROR: Unknown data type. Check directory contents.")
                return

            # 2. Calculate Display Levels
            if mip_original is not None:
                vmin_original, vmax_original = np.percentile(mip_original, [1, 99.9])
                if vmin_original >= vmax_original:
                    vmax_original = np.max(mip_original)

            # --- 3. AUTO-DETECT METADATA ---
            print("\n--- Auto-detecting Metadata ---")

            # A. Find Metadata File
            metadata_file = None
            if base_name_source:
                # OPM metadata usually in parent dir: base_name + "_metadata.txt"
                possible_meta = source_processed_dir.parent / (
                    base_name_source + "_metadata.txt"
                )
                if possible_meta.exists():
                    metadata_file = possible_meta

            # B. Parse Z-Step
            if metadata_file:
                try:
                    found_z = opym.metadata.parse_z_step(
                        metadata_file, default_z_step=default_z_step
                    )
                    if found_z:
                        default_z_step = found_z
                        print(f"✅ Found Z-Step in metadata: {default_z_step} µm")
                    else:
                        print(
                            f"⚠️ Could not parse Z-step from {metadata_file.name}. "
                            "Keeping default ({default_z_step} µm)."
                        )
                except Exception:
                    print(
                        "⚠️ Error reading metadata. Keeping default Z "
                        "({default_z_step} µm)."
                    )
            else:
                print(
                    f"ℹ️ No metadata file found. Keeping default Z ({default_z_step} "
                    "µm)."
                )

            print(f"\n✅ {data_type_source} data loaded successfully.")
            print(f"   Base Name: {base_name_source}")
            print(
                f"   Defaults for next step -> Z: {default_z_step}, Angle: "
                "{default_angle}"
            )
            print("You can now run Cell 1.")

        except FileNotFoundError as e:
            print(f"\n❌ ERROR: Could not load data: {e}")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)
            traceback.print_exc(file=sys.stderr)


load_button.on_click(on_load_button_clicked)
display(widgets.VBox([dir_chooser, preview_channel_widget, load_button, load_output]))

In [ ]:
# --- Cell 1: Interactive Parameter Tuner (v4) ---

import time

import ipywidgets as widgets
import numpy as np
from IPython.display import display

# --- Global to hold parameters for viewer cells ---
processed_viewer_params = None

# --- Create Parameter Sliders (RENAMED v4) ---
angle_slider_v4 = widgets.FloatSlider(
    value=default_angle,
    min=10.0,
    max=40.0,
    step=0.01,
    description="Sheet Angle:",
    readout_format=".2f",
    style={"description_width": "initial"},
)
z_step_slider_v4 = widgets.FloatSlider(
    value=default_z_step,
    min=0.05,
    max=5.0,
    step=0.05,
    description="Z-Step (um):",
    readout_format=".2f",
    style={"description_width": "initial"},
)
xy_size_slider_v4 = widgets.FloatSlider(
    value=default_xy_size,
    min=0.05,
    max=0.2,
    step=0.001,
    description="XY Pixel Size (um):",
    readout_format=".3f",
    style={"description_width": "initial"},
)

# --- NEW: Reverse Z Checkbox ---
reverse_z_checkbox_v4 = widgets.Checkbox(
    value=False,
    description="Reverse Z-Stack",
    indent=False,
)

# --- Create Buttons and Output Areas (RENAMED v4) ---
run_view_button_v4 = widgets.Button(
    description="Run & View Processed Stack", button_style="success"
)
log_output_v4 = widgets.Output(
    layout={"border": "1px solid black", "height": "200px", "overflow_y": "scroll"}
)
view_output_v4 = widgets.Output()


# --- Define Button Click Handler (RENAMED v4) ---
def on_run_view_button_clicked_v4(b):
    global processed_viewer_params
    run_view_button_v4.disabled = True

    with log_output_v4:
        clear_output(wait=True)
    with view_output_v4:
        clear_output(wait=True)

    try:
        if (
            source_processed_dir is None
            or mip_original is None
            or stack_original is None
        ):
            with log_output_v4:
                print("❌ ERROR: Source data not loaded. Please run Cell 0 first.")
            return

        if data_type_source == "UNKNOWN":
            with log_output_v4:
                print(
                    "❌ ERROR: Data type is UNKNOWN. "
                    "Please re-run Cell 0 and select a valid directory."
                )
            return

        # 1. Get all parameters from NEW widgets
        angle = angle_slider_v4.value
        z_step = z_step_slider_v4.value
        xy_size = xy_size_slider_v4.value
        reverse_z = reverse_z_checkbox_v4.value  # <-- Get value

        with log_output_v4:
            print(f"--- [RUNNING v4] Processing {data_type_source} data with: ---")
            print(f"  Angle={angle}, Z-Step={z_step}, XY-Size={xy_size}")
            print(f"  Reverse Z: {reverse_z}")

        # 2. Create unique directory name for this run
        # Include 'rev' in name if reversed so you can compare
        rev_str = "_rev" if reverse_z else ""
        dsr_dir_name = f"DSR_a{angle:.2f}_z{z_step:.2f}_xy{xy_size:.3f}{rev_str}"
        ds_dir_name = f"DS_a{angle:.2f}_z{z_step:.2f}_xy{xy_size:.3f}{rev_str}"

        # 3. Collect all parameters into a dict
        processing_params = {
            "ds_dir_name": ds_dir_name,
            "dsr_dir_name": dsr_dir_name,
            "sheet_angle_deg": angle,
            "z_step_um": z_step,
            "xy_pixel_size": xy_size,
            "save_mip": True,
            "reverse_z": reverse_z,  # <-- Pass to function
        }

        if data_type_source == "OPM":
            print("--- Running OPM processing (opym.run_petakit_processing) ---")
            opym.run_petakit_processing(source_processed_dir, **processing_params)
        elif data_type_source == "LLSM":
            print("--- Running LLSM processing (opym.run_llsm_petakit_processing) ---")
            opym.run_llsm_petakit_processing(source_processed_dir, **processing_params)

        # 4. Load the processed stack
        output_path = source_processed_dir / dsr_dir_name

        # --- FIX: Wait for filesystem sync ---
        if not output_path.exists():
            with log_output_v4:
                print(f"⏳ Waiting for filesystem sync: {output_path.name}...")
            # Poll for up to 10 seconds (20 * 0.5s)
            for _ in range(20):
                if output_path.exists():
                    break
                time.sleep(0.5)
        # -------------------------------------

        with log_output_v4:
            print(f"✅ Processing complete. Loading results from: {output_path.name}")

        (
            get_stack_proc,
            t_min_proc,
            t_max_proc,
            c_min_proc,
            c_max_proc,
            z_max_proc,
            y_proc,
            x_proc,
            bn_proc,
        ) = opym.load_tiff_series(output_path)

        processed_viewer_params = (
            get_stack_proc,
            t_max_proc,
            z_max_proc,
            c_max_proc,
            y_proc,
            x_proc,
        )

        preview_channel_to_load = preview_channel_widget.value

        if not c_min_proc <= preview_channel_to_load <= c_max_proc:
            with log_output_v4:
                print(
                    f"⚠️ Warning: Preview channel {preview_channel_to_load} "
                    f"not in processed range ({c_min_proc}-{c_max_proc})."
                )
                print(f"   Defaulting to processed C={c_min_proc}.")
            preview_channel_to_load = c_min_proc

        with log_output_v4:
            print(
                f"   Loading processed T={t_min_proc}, "
                f"C={preview_channel_to_load} for MIP."
            )

        stack_processed = get_stack_proc(t_min_proc, preview_channel_to_load)

        # 5. Calculate all Orthoview MIPS
        mip_original_xy = mip_original
        mip_original_xz = np.max(stack_original, axis=1)
        mip_original_yz = np.max(stack_original, axis=2)

        mip_processed_xy = np.max(stack_processed, axis=0)
        mip_processed_xz = np.max(stack_processed, axis=1)
        mip_processed_yz = np.max(stack_processed, axis=2)

        vmin_proc, vmax_proc = np.percentile(mip_processed_xy, [1, 99.9])
        if vmin_proc >= vmax_proc:
            vmax_proc = np.max(mip_processed_xy)

        # 6. Display the 2x3 Orthoview results
        with view_output_v4:
            fig, axes = plt.subplots(2, 3, figsize=(15, 8))

            axes[0, 0].imshow(
                mip_original_xy, cmap="gray", vmin=vmin_original, vmax=vmax_original
            )
            axes[0, 0].set_title(f"Original XY (Shape: {mip_original_xy.shape})")
            axes[0, 0].axis("off")

            axes[0, 1].imshow(
                mip_original_xz, cmap="gray", vmin=vmin_original, vmax=vmax_original
            )
            axes[0, 1].set_title(f"Original XZ (Shape: {mip_original_xz.shape})")
            axes[0, 1].axis("off")
            axes[0, 1].set_aspect("auto")

            axes[0, 2].imshow(
                mip_original_yz, cmap="gray", vmin=vmin_original, vmax=vmax_original
            )
            axes[0, 2].set_title(f"Original YZ (Shape: {mip_original_yz.shape})")
            axes[0, 2].axis("off")
            axes[0, 2].set_aspect("auto")

            axes[1, 0].imshow(
                mip_processed_xy, cmap="gray", vmin=vmin_proc, vmax=vmax_proc
            )
            axes[1, 0].set_title(f"Processed XY (Shape: {mip_processed_xy.shape})")
            axes[1, 0].axis("off")

            axes[1, 1].imshow(
                mip_processed_xz, cmap="gray", vmin=vmin_proc, vmax=vmax_proc
            )
            axes[1, 1].set_title(f"Processed XZ (Shape: {mip_processed_xz.shape})")
            axes[1, 1].axis("off")
            axes[1, 1].set_aspect("auto")

            axes[1, 2].imshow(
                mip_processed_yz, cmap="gray", vmin=vmin_proc, vmax=vmax_proc
            )
            axes[1, 2].set_title(f"Processed YZ (Shape: {mip_processed_yz.shape})")
            axes[1, 2].axis("off")
            axes[1, 2].set_aspect("auto")

            plt.tight_layout()
            plt.show()

            with log_output_v4:
                print("\n✅ Orthoviews displayed.")
                print(
                    "You can now run Cell 2 or 3 to view the processed stack interactively."
                )

    except Exception as e:
        with log_output_v4:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)
            traceback.print_exc(file=sys.stderr)
    finally:
        # Re-enable the button
        run_view_button_v4.disabled = False


# --- Link button and display UI ---
run_view_button_v4.on_click(on_run_view_button_clicked_v4)

param_box_v4 = widgets.VBox(
    [angle_slider_v4, z_step_slider_v4, xy_size_slider_v4, reverse_z_checkbox_v4]
)

# Display the NEW widgets
display(widgets.VBox([param_box_v4, run_view_button_v4, log_output_v4, view_output_v4]))

In [ ]:
# --- Cell 1.5: View Raw (Pre-Deskew) Data ---
# This loads the data from 'processed_tiff_series_split' for inspection.

try:
    if source_processed_dir is None:
        raise NameError("source_processed_dir not defined.")

    print(f"--- Loading Raw Input Data from: {source_processed_dir.name} ---")

    # 1. Load the raw tiff series
    (
        get_stack_raw,
        t_min_raw,
        t_max_raw,
        c_min_raw,
        c_max_raw,
        z_max_raw,
        y_raw,
        x_raw,
        bn_raw,
    ) = opym.load_tiff_series(source_processed_dir)

    # 2. Pack parameters for the viewer
    raw_viewer_params = (
        get_stack_raw,
        t_max_raw,
        z_max_raw,
        c_max_raw,
        y_raw,
        x_raw,
    )

    print(
        f"✅ Loaded {bn_raw}. Shape: (T={t_max_raw + 1}, Z={z_max_raw + 1}, Y={y_raw}, X={x_raw})"
    )

    # 3. Launch Viewer
    opym.composite_viewer(*raw_viewer_params)

except NameError:
    print(
        "❌ ERROR: Please run Cell 0 and Cell 1 first to define the source directory."
    )
except FileNotFoundError as e:
    print(f"❌ ERROR: Could not find files: {e}")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 2: Launch the composite viewer ---
# (Run this cell after running Cell 1)

try:
    if processed_viewer_params is None:
        raise NameError
    opym.composite_viewer(*processed_viewer_params)
except NameError:
    print(
        "❌ ERROR: 'processed_viewer_params' not found. Please run Cell 1 to process "
        "data first."
    )
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 3: Fine-Tune Angle via Bead Centroid Analysis ---
# (Run this after Cell 1 & 2 to calculate precise angle corrections)

import ipywidgets as widgets
import numpy as np
from IPython.display import display
from matplotlib.backend_bases import MouseButton
from matplotlib.widgets import RectangleSelector
from scipy.ndimage import center_of_mass
from sklearn.linear_model import LinearRegression

# --- 1. Retrieve Data from Global Params ---
stack_for_analysis = None

try:
    # --- FIX: Assign to local variable for strict type checking ---
    # Pylance handles type narrowing on local variables much better than globals.
    params = processed_viewer_params

    if params is None:
        raise NameError("processed_viewer_params is None")

    # Now access the local 'params', which the checker knows is not None
    # params structure: (get_stack_proc, t_max, z_max, c_max, y, x)
    get_stack_fn = params[0]
    c_max_val = params[3]

    # Load T=0 and the selected preview channel (or default to 0)
    target_c = (
        preview_channel_widget.value if "preview_channel_widget" in globals() else 0
    )

    # Validate channel bounds
    if target_c > c_max_val:
        target_c = 0

    print(f"✅ Loaded data for analysis (Channel {target_c}).")
    stack_for_analysis = get_stack_fn(0, target_c)

except NameError:
    print("❌ ERROR: 'processed_viewer_params' not found.")
    print("   Please run Cell 1 (Run & View) to process data first.")
except Exception as e:
    print(f"❌ Error loading data: {e}")


# --- Global State for Selection ---
bead_selector = None
bead_roi = None

# --- Widgets ---
analyze_button = widgets.Button(
    description="Analyze Bead", button_style="primary", icon="crosshairs"
)
output_area = widgets.Output()


def perform_bead_analysis(b):
    with output_area:
        clear_output(wait=True)

        # 1. Get ROI
        if (
            bead_selector is None
            or not hasattr(bead_selector, "rois")
            or not bead_selector.rois
        ):
            print("⚠️ Please draw a box around a SINGLE bead in the plot above.")
            return

        # Take the last drawn box
        roi = bead_selector.rois[-1]
        # ROI format is (x_min, x_max, y_min, y_max)
        x1, x2, y1, y2 = map(int, roi)

        # 2. Crop the stack (Z, Y, X)
        bead_vol = stack_for_analysis[:, y1:y2, x1:x2]  # type: ignore

        if bead_vol.size == 0:
            print("❌ Error: Selected ROI is empty.")
            return

        print(f"--- Analyzing Bead ROI: X[{x1}:{x2}], Y[{y1}:{y2}] ---")
        print(f"    Volume shape: {bead_vol.shape}")

        # 3. Calculate Centroids per Z-slice
        z_coords = []
        x_centroids = []
        y_centroids = []

        # Threshold to avoid background noise (20% of max in crop)
        thresh = np.max(bead_vol) * 0.2

        for z in range(bead_vol.shape[0]):
            plane = bead_vol[z, :, :]
            if np.max(plane) > thresh:
                # center_of_mass returns (y, x)
                cy, cx = center_of_mass(plane)
                z_coords.append(z)
                x_centroids.append(cx)
                y_centroids.append(cy)

        if len(z_coords) < 5:
            print(
                "⚠️ Not enough signal to track bead. Try a brighter bead or check "
                "Z-range."
            )
            return

        # Convert to numpy arrays
        z_arr = np.array(z_coords)
        x_arr = np.array(x_centroids)
        y_arr = np.array(y_centroids)

        # 4. Physical Units Conversion
        # Retrieve params from Cell 1 Sliders
        curr_z_step = z_step_slider.value
        curr_xy_size = xy_size_slider.value
        curr_angle = angle_slider.value

        z_microns = z_arr * curr_z_step
        x_microns = x_arr * curr_xy_size
        y_microns = y_arr * curr_xy_size

        # 5. Fit Lines (Drift Calculation)
        # Reshape for sklearn: (n_samples, n_features)
        reg_x = LinearRegression().fit(z_microns.reshape(-1, 1), x_microns)
        slope_x = reg_x.coef_[0]  # microns drift per micron Z

        reg_y = LinearRegression().fit(z_microns.reshape(-1, 1), y_microns)
        slope_y = reg_y.coef_[0]

        # 6. Calculate Angle Correction
        # Geometry: real_tan = current_tan + drift_slope
        current_tan = np.tan(np.deg2rad(curr_angle))

        # NOTE: The sign depends on the deskew direction.
        # If bead drifts +X as Z increases, we usually need a larger angle.
        new_tan = current_tan + slope_x
        suggested_angle = np.rad2deg(np.arctan(new_tan))
        angle_diff = suggested_angle - curr_angle

        # 7. Visualization
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))

        # Plot A: X-Drift
        axes[0].scatter(z_microns, x_microns, alpha=0.5, label="Data", s=10)
        axes[0].plot(
            z_microns,
            reg_x.predict(z_microns.reshape(-1, 1)),
            "r-",
            label=f"Fit (m={slope_x:.4f})",
        )
        axes[0].set_xlabel("Z Position (µm)")
        axes[0].set_ylabel("X Centroid (µm)")
        axes[0].set_title("X Drift (Deskew Error)")
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

        # Plot B: Y-Drift
        axes[1].scatter(z_microns, y_microns, alpha=0.5, s=10)
        axes[1].plot(
            z_microns,
            reg_y.predict(z_microns.reshape(-1, 1)),
            "r-",
            label=f"Fit (m={slope_y:.4f})",
        )
        axes[1].set_xlabel("Z Position (µm)")
        axes[1].set_ylabel("Y Centroid (µm)")
        axes[1].set_title("Y Drift (Rotation/Alignment)")
        axes[1].grid(True, alpha=0.3)

        # Plot C: Visual XZ Projection with Centerline
        bead_mip_y = np.max(bead_vol, axis=1)
        aspect_ratio = curr_z_step / curr_xy_size
        axes[2].imshow(bead_mip_y, cmap="magma", aspect=aspect_ratio, origin="lower")

        # Overlay the fitted line
        z_plot_pix = np.array([0, bead_vol.shape[0] - 1])
        z_plot_um = z_plot_pix * curr_z_step
        x_pred_um = reg_x.predict(z_plot_um.reshape(-1, 1))
        x_pred_pix = x_pred_um / curr_xy_size

        axes[2].plot(
            x_pred_pix, z_plot_pix, "w--", linewidth=1.5, label="Detected Axis"
        )
        axes[2].set_title("Bead Side View (XZ)")
        axes[2].set_xlabel("X (pixels)")
        axes[2].set_ylabel("Z (slices)")
        axes[2].legend()

        plt.tight_layout()
        plt.show()

        # 8. Report
        print("\n" + "=" * 45)
        print("🔎 ANALYSIS RESULTS")
        print("=" * 45)
        print(f"Current Angle:    {curr_angle:.2f}°")
        print(f"X-Drift Slope:    {slope_x:.5f} (µm shift per µm Z)")
        print(f"Y-Drift Slope:    {slope_y:.5f}")
        print("-" * 45)
        print("💡 SUGGESTED CORRECTION:")
        print(f"   New Angle:     {suggested_angle:.2f}°")
        print(f"   Difference:    {angle_diff:+.2f}°")
        print("=" * 45)
        print("👉 Update the 'Sheet Angle' slider in Cell 1 to the New Angle")
        print("   and click 'Run & View' again to verify.")


# --- Initialize UI ---
if stack_for_analysis is not None:
    # Calculate MIP for selection
    mip_for_selection = np.max(stack_for_analysis, axis=0)
    vmin, vmax = np.percentile(mip_for_selection, [1, 99.5])

    print("Draw a box around ONE bead to analyze:")

    fig_sel = plt.figure(figsize=(8, 8))
    ax_sel = fig_sel.add_subplot(111)
    ax_sel.imshow(mip_for_selection, cmap="gray", vmin=vmin, vmax=vmax)
    ax_sel.set_title("Draw Box Around Bead -> Click 'Analyze Bead'")

    class SimpleSelector:
        def __init__(self):
            self.rois = []
            self.rs = RectangleSelector(
                ax_sel,
                self.line_select_callback,
                useblit=True,
                button=[MouseButton.LEFT],
                minspanx=5,
                minspany=5,
                spancoords="data",
                interactive=True,
            )

        def line_select_callback(self, eclick, erelease):
            x1, y1 = eclick.xdata, eclick.ydata
            x2, y2 = erelease.xdata, erelease.ydata
            self.rois = [(min(x1, x2), max(x1, x2), min(y1, y2), max(y1, y2))]

    bead_selector = SimpleSelector()
    plt.show()

    analyze_button.on_click(perform_bead_analysis)
    display(widgets.VBox([analyze_button, output_area]))